In [11]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd

### 1. HuggingFace에 업로드된 임베딩 모델 (bge-m3)을 사용해보자.
- bge-m3는 BAAI (베이징 인공지능 연구소)에서 개발한 다기능 임베딩 모델로, 한국어를 포함한 다국어의 포괄적 지원이 가능한 특징이 있다.
- 2024년 2월 기준, 한국어 벤치마크에서 SOTA 성능을 달성한 바 있다.
- Reference: https://arxiv.org/pdf/2402.03216 (ACL 2024)

In [4]:
model = SentenceTransformer('BAAI/bge-m3')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\yechan_amd\anaconda3\envs\rag\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yechan_amd\.cache\huggingface\hub\models--BAAI--bge-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [5]:
model.encode('종현이는 수연이를 사모한다.')

array([-0.05094872,  0.02595654, -0.07525721, ...,  0.0033758 ,
       -0.04616366,  0.02705512], shape=(1024,), dtype=float32)

In [7]:
model.encode('종현이는 욱과 구 사이에서 마치 등 터진 새우와 같았다.')

array([ 0.01912978, -0.0017126 , -0.05473248, ...,  0.00775997,
       -0.03408086, -0.02759308], shape=(1024,), dtype=float32)

In [8]:
def get_embedding(sentence):
    return list(model.encode(sentence))

In [12]:
data = ['저는 배가 고파요',
        '저기 배가 지나가네요',
        '굶어서 허기가 지네요',
        '허기 워기라는 게임이 있는데 즐거워',
        '스팀에서 재밌는 거 해야지',
        '스팀에어프라이어로 연어구이 해먹을거야',
        '종현은 수연을 좋아한다']
df = pd.DataFrame(data, columns=['text'])
df

,text
0,저는 배가 고파요
1,저기 배가 지나가네요
2,굶어서 허기가 지네요
3,허기 워기라는 게임이 있는데 즐거워
4,스팀에서 재밌는 거 해야지
5,스팀에어프라이어로 연어구이 해먹을거야
6,종현은 수연을 좋아한다


In [13]:
df['embedding'] = df['text'].apply(get_embedding)

In [14]:
df

,text,embedding
0,저는 배가 고파요,"[0.021626636, 0.009280884, -0.052108202, -0.01..."
1,저기 배가 지나가네요,"[-0.0033455137, 0.008186357, -0.07204161, -0.0..."
2,굶어서 허기가 지네요,"[0.0004956166, 0.027304467, -0.060060877, -0.0..."
3,허기 워기라는 게임이 있는데 즐거워,"[-0.01299883, 0.019039104, -0.071179815, -0.00..."
4,스팀에서 재밌는 거 해야지,"[-0.035401918, 0.002258152, -0.049874898, -0.0..."
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.02898656, 0.0040246467, -0.032376584, -0.0..."
6,종현은 수연을 좋아한다,"[-0.03163556, -0.012156947, -0.049750816, -0.0..."


In [15]:
def cos_sim(a, b):
    return np.dot(a, b) / np.linalg.norm(a) * np.linalg.norm(b)

In [17]:
def return_answer_candidate(df, query, top_k=3):
    query_embedding = get_embedding(query)
    df['similarity'] = df['embedding'].apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))
    results = df.sort_values(by=['similarity'], ascending=False, ignore_index=True)[:top_k]
    return results

In [20]:
sim_result = return_answer_candidate(df, '아무 것도 안 먹었더니 꼬르륵 소리가 나네')
sim_result

,text,embedding,similarity
0,굶어서 허기가 지네요,"[0.0004956166, 0.027304467, -0.060060877, -0.0...",0.566914
1,저는 배가 고파요,"[0.021626636, 0.009280884, -0.052108202, -0.01...",0.498033
2,허기 워기라는 게임이 있는데 즐거워,"[-0.01299883, 0.019039104, -0.071179815, -0.00...",0.474015


### 2. OpenAI가 유료로 제공하는 API 형태의 임베딩 모델 (text-embedding-ada-002)을 사용해보자.
- text-embedding-ada-002: 2022년 12월 출시된 비교적 저렴한 API 모델 (dim: 1536)
- text-similarity-babbage-001 (dim: 2048)
- ...

In [33]:
from openai import OpenAI
import openai

OPENAI_API_KEY = '당신의 키 값을 입력하세요. MLV 계정 키 값 발급/사용 금지!!!'

openai.api_key = OPENAI_API_KEY

In [35]:
client = OpenAI(
    api_key=OPENAI_API_KEY,
)

In [31]:
def get_embedding_by_openai(sentence, model='text-embedding-ada-002'):
    res = client.embeddings.create(
        input=sentence,
        model=model,
    )
    return res.data[0].embedding

In [36]:
get_embedding_by_openai('종현이는 수연이를 사모한다.')

[-0.0135382991284132,
 -0.02451600879430771,
 0.024542272090911865,
 -0.0030415214132517576,
 -0.028100835159420967,
 0.028547296300530434,
 -0.014588798396289349,
 0.005229512695223093,
 0.0004534378240350634,
 -0.00917873065918684,
 0.004874969832599163,
 0.02733922377228737,
 -0.0062504662200808525,
 -0.026301857084035873,
 -0.003338615642860532,
 -0.00060198490973562,
 0.025710951536893845,
 -0.006706776563078165,
 0.0065032425336539745,
 -0.03154121711850166,
 -0.012015076354146004,
 -0.009507011622190475,
 -0.0015273262979462743,
 0.007478236220777035,
 -0.013091837987303734,
 0.03823814541101456,
 0.006069911643862724,
 -0.022112993523478508,
 0.017490800470113754,
 0.002563872840255499,
 0.01743827573955059,
 -0.0027378618251532316,
 0.029886683449149132,
 -0.002606549533084035,
 -0.01162770576775074,
 -0.0007242694846354425,
 0.014667585492134094,
 0.012612547725439072,
 0.0012679845094680786,
 0.000984021695330739,
 -0.00398861151188612,
 -0.013111534528434277,
 -0.0049898680

In [37]:
len(get_embedding_by_openai('종현이는 수연이를 사모한다.'))

1536

In [38]:
df

,text,embedding,similarity
0,저는 배가 고파요,"[0.021626636, 0.009280884, -0.052108202, -0.01...",0.498033
1,저기 배가 지나가네요,"[-0.0033455137, 0.008186357, -0.07204161, -0.0...",0.436127
2,굶어서 허기가 지네요,"[0.0004956166, 0.027304467, -0.060060877, -0.0...",0.566914
3,허기 워기라는 게임이 있는데 즐거워,"[-0.01299883, 0.019039104, -0.071179815, -0.00...",0.474015
4,스팀에서 재밌는 거 해야지,"[-0.035401918, 0.002258152, -0.049874898, -0.0...",0.392223
5,스팀에어프라이어로 연어구이 해먹을거야,"[-0.02898656, 0.0040246467, -0.032376584, -0.0...",0.404476
6,종현은 수연을 좋아한다,"[-0.03163556, -0.012156947, -0.049750816, -0.0...",0.315699


In [39]:
df['embedding'] = df['text'].apply(get_embedding_by_openai)

In [40]:
def return_answer_candidate_by_openai(df, query, top_k=3):
    query_embedding = get_embedding_by_openai(query)
    df['similarity'] = df['embedding'].apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))
    results = df.sort_values(by=['similarity'], ascending=False, ignore_index=True)[:top_k]
    return results

In [41]:
sim_result = return_answer_candidate_by_openai(df, '아무 것도 안 먹었더니 꼬르륵 소리가 나네')
sim_result

,text,embedding,similarity
0,굶어서 허기가 지네요,"[-0.006181030999869108, -0.0069507937878370285...",0.836694
1,스팀에어프라이어로 연어구이 해먹을거야,"[-0.002138908952474594, -0.030034277588129044,...",0.817397
2,저는 배가 고파요,"[-0.01663736067712307, -0.02178889885544777, 0...",0.813161


In [42]:
sim_result = return_answer_candidate_by_openai(df, '종현은 성헌을 그윽하게 본다.')
sim_result

,text,embedding,similarity
0,종현은 수연을 좋아한다,"[-0.013112376444041729, -0.017058944329619408,...",0.913418
1,굶어서 허기가 지네요,"[-0.006181030999869108, -0.0069507937878370285...",0.822173
2,스팀에서 재밌는 거 해야지,"[-0.016108456999063492, -0.014401600696146488,...",0.796376
